基于**Learning both Weights and Connections for Efficient Neural Network**论文的progan模型修剪

In [1]:
import pandas as pd
#import dnnlib
#import dnnlib.tflib as tflib
import tfutil
import dataset
import misc
import numpy as np
import config 
import tensorflow as tf
import os

import pickle

path = 'model/'+'pgan-wm3000-20000'
tfutil.init_tf(config.tf_config)
with tf.device('/gpu:0'):
    network_pkl = misc.locate_network_pkl(path+'.pkl')
    print('Loading networks from "%s"...' % network_pkl)
    G, D, Gs = misc.load_pkl(network_pkl)

W0727 12:37:48.815481 139634556143424 deprecation_wrapper.py:119] From /tf/notebooks/progressive_growing_of_gans-master/tfutil.py:56: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0727 12:37:48.816479 139634556143424 deprecation_wrapper.py:119] From /tf/notebooks/progressive_growing_of_gans-master/tfutil.py:57: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.

W0727 12:37:48.832114 139634556143424 deprecation_wrapper.py:119] From /tf/notebooks/progressive_growing_of_gans-master/tfutil.py:65: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0727 12:37:48.832686 139634556143424 deprecation_wrapper.py:119] From /tf/notebooks/progressive_growing_of_gans-master/tfutil.py:72: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0727 12:37:49.744889 139634556143424 deprecation_wrapper.py:119] From /tf/notebooks/progressive_growing_of_

Loading networks from "model/pgan-wm3000-20000.pkl"...


In [2]:
# dict1 = Gs.__getstate__()
# for item in dict1['variables']:
#     print(item[0])
# for item in G.vars:
#     print(item)

In [3]:
# import dnnlib.tflib.tfutil as tfutil
import numpy as np
def threshold_calculation(w_conv, layer_name,prune_rate):
   # w_conv = item[1] # weight四维参数
    w_parameter = sorted(abs(w_conv.flatten()))
    print(layer_name)
#     print('参数裁剪前总和',sum(w_parameter))
#     print('参数个数',len(w_parameter))
    print('裁剪个数',int(len(w_parameter) * prune_rate))
    threshold = w_parameter[int(len(w_parameter) * prune_rate)]
    print('阈值',threshold)
    return threshold
    

def prune_main(G, D, Gs,prune_rate):
    print('---------',prune_rate,'---------')
#     dict_Gs = Gs.__getstate__()
    dict_variables = G.vars#dict_Gs['variables']
    
    for item in dict_variables:
#         print(item)
        #if (item.find('Conv') == -1 or item.find('up') != -1):
        if (item.find('Conv') == -1):
            continue    

        if '/weight' in item:              
            w_conv = Gs.get_var(item) # weight四维参数
            threshold = threshold_calculation(w_conv , item, prune_rate)

            for item1 in range(len(w_conv)):
                for item2 in range(len(w_conv[0])):
                    for item3 in range(len(w_conv[0][0])):
                        for item4 in range(len(w_conv[0][0][0])):
                            if abs(w_conv[item1][item2][item3][item4]) < threshold:
                                w_conv[item1][item2][item3][item4] = 0
           # w_parameter = sorted(abs(w_conv.flatten()))  
            Gs.set_var(item, w_conv)
            G.set_var(item, w_conv)
#             print('参数裁剪后总和',sum(w_parameter))
        if '/mod_weight' in item:              
            w_conv = Gs.get_var(item)
            threshold = threshold_calculation(w_conv,item, prune_rate)

            for item1 in range(len(w_conv)):
                for item2 in range(len(w_conv[0])):
                    if abs(w_conv[item1][item2]) < threshold:
                        w_conv[item1][item2] = 0    
#             print('参数裁剪后总和',sum(w_parameter))           

            
    pkl = os.path.join('model/','test_%d.pkl'%(prune_rate*100))
    misc.save_pkl((G, D, Gs), pkl)  
    print('--------模型model/','test_%d.pkl'%(prune_rate*100),'保存完毕--------')
    

In [4]:
# prune_rates = [0.1,0.3,0.5,0.7,0.9] #修剪的概率
prune_rates = [0,0.1,0.9]
# prune_rates = [0.9]
for prune_rate in prune_rates:
    prune_main(G, D, Gs,prune_rate)

--------- 0 ---------
4x4/Conv/weight
裁剪个数 0
阈值 1.3969839e-08
8x8/Conv0_up/weight
裁剪个数 0
阈值 1.4416873e-06
8x8/Conv1/weight
裁剪个数 0
阈值 1.6763806e-08
16x16/Conv0_up/weight
裁剪个数 0
阈值 0.0
16x16/Conv1/weight
裁剪个数 0
阈值 2.2199747e-06
32x32/Conv0_up/weight
裁剪个数 0
阈值 1.1920929e-07
32x32/Conv1/weight
裁剪个数 0
阈值 2.1979213e-07
--------模型model/ test_0.pkl 保存完毕--------
--------- 0.1 ---------
4x4/Conv/weight
裁剪个数 235929
阈值 0.14334361
8x8/Conv0_up/weight
裁剪个数 235929
阈值 0.14468355
8x8/Conv1/weight
裁剪个数 235929
阈值 0.14387304
16x16/Conv0_up/weight
裁剪个数 235929
阈值 0.14402883
16x16/Conv1/weight
裁剪个数 235929
阈值 0.14296535
32x32/Conv0_up/weight
裁剪个数 235929
阈值 0.14263958
32x32/Conv1/weight
裁剪个数 235929
阈值 0.14081691
--------模型model/ test_10.pkl 保存完毕--------
--------- 0.9 ---------
4x4/Conv/weight
裁剪个数 2123366
阈值 1.8782599
8x8/Conv0_up/weight
裁剪个数 2123366
阈值 1.8984601
8x8/Conv1/weight
裁剪个数 2123366
阈值 1.8878767
16x16/Conv0_up/weight
裁剪个数 2123366
阈值 1.890611
16x16/Conv1/weight
裁剪个数 2123366
阈值 1.8893297
32x32/Conv0_up

## 模型评估
- is
- fid

In [8]:
import pandas as pd
import tfutil
import dataset
import misc
import numpy as np
import config 
import tensorflow as tf
import os
import pickle


import config
import time

def evaluate_metrics(snapshot_pkls,dataset_cfg,metrics):
    metric_class_names = {
        'swd':      'metrics.sliced_wasserstein.API',
        'fid':      'metrics.frechet_inception_distance.API',
        'is':       'metrics.inception_score.API',
        'msssim':   'metrics.ms_ssim.API',
    }
     
    pklpath = 'model'
    
    num_images=50000
    real_passes=1
    
    # 初始化模型
#     network_pkl = misc.locate_network_pkl(pklpath+'.pkl')
#     snapshot_pkls = ['model/test_0.pkl','model/test_10.pkl','model/test_20.pkl','model/test_30.pkl',
#                      'model/test_40.pkl','model/test_50.pkl','model/test_60.pkl','model/test_70.pkl',
#                      'model/test_80.pkl','model/test_90.pkl']
    
    # 加载数据
    dataset_obj = dataset.load_dataset(data_dir=config.data_dir, **dataset_cfg)
    mirror_augment = True
    minibatch_size = np.clip(8192 // dataset_obj.shape[1], 4, 256)
    
    # 初始化metrics
#     metrics=['is','fid']
    metric_objs = []
    for name in metrics:
        class_name = metric_class_names.get(name, name)
        print('Initializing %s...' % class_name)
        class_def = tfutil.import_obj(class_name)
        image_shape = [3] + dataset_obj.shape[1:]
        print(image_shape)
        obj = class_def(num_images=num_images, image_shape=image_shape, image_dtype=np.uint8, minibatch_size=minibatch_size)
        tfutil.init_uninited_vars()
        mode = 'warmup'
        obj.begin(mode)
        for idx in range(10):
            obj.feed(mode, np.random.randint(0, 256, size=[minibatch_size]+image_shape, dtype=np.uint8))
        obj.end(mode)
        metric_objs.append(obj)

    # Print table header.
    print()
    print('%-10s%-12s' % ('Snapshot', 'Time_eval'), end='')
    for obj in metric_objs:
        for name, fmt in zip(obj.get_metric_names(), obj.get_metric_formatting()):
            print('%-*s' % (len(fmt % 0), name), end='')
    print()
    print('%-10s%-12s' % ('---', '---'), end='')
    for obj in metric_objs:
        for fmt in obj.get_metric_formatting():
            print('%-*s' % (len(fmt % 0), '---'), end='')
    print()

    # Feed in reals.
    for title, mode in [('Reals', 'reals'), ('Reals2', 'fakes')][:real_passes]:
        print('%-10s' % title, end='')
        time_begin = time.time()
        labels = np.zeros([num_images, dataset_obj.label_size], dtype=np.float32)
        [obj.begin(mode) for obj in metric_objs]
        ############  
        for begin in range(0, num_images, minibatch_size):
            end = min(begin + minibatch_size, num_images)
            images, labels[begin:end] = dataset_obj.get_minibatch_np(end - begin)
            if mirror_augment:
                images = misc.apply_mirror_augment(images)
            if images.shape[1] == 1:
                images = np.tile(images, [1, 3, 1, 1]) # grayscale => RGB
            [obj.feed(mode, images) for obj in metric_objs]
        results = [obj.end(mode) for obj in metric_objs]
        print('%-12s' % misc.format_time(time.time() - time_begin), end='')
        for obj, vals in zip(metric_objs, results):
            for val, fmt in zip(vals, obj.get_metric_formatting()):
                print(fmt % val, end='')
        ############        
        print()
        
    # Evaluate each network snapshot.
    for snapshot_idx, snapshot_pkl in enumerate(reversed(snapshot_pkls)):
        prefix = 'test_'; postfix = '.pkl'
        snapshot_name = os.path.basename(snapshot_pkl)
        assert snapshot_name.startswith(prefix) and snapshot_name.endswith(postfix)
        snapshot_kimg = int(snapshot_name[len(prefix) : -len(postfix)])

        print('%-10d' % snapshot_kimg, end='')
        mode ='fakes'
        [obj.begin(mode) for obj in metric_objs]
        time_begin = time.time()
        with tf.Graph().as_default(), tfutil.create_session(config.tf_config).as_default():
            G, D, Gs = misc.load_pkl(snapshot_pkl)
            for begin in range(0, num_images, minibatch_size):
                end = min(begin + minibatch_size, num_images)
                latents = misc.random_latents(end - begin, Gs)
                images = Gs.run(latents, labels[begin:end], num_gpus=1, out_mul=127.5, out_add=127.5, out_dtype=np.uint8)
                if images.shape[1] == 1:
                    images = np.tile(images, [1, 3, 1, 1]) # grayscale => RGB
                [obj.feed(mode, images) for obj in metric_objs]
        results = [obj.end(mode) for obj in metric_objs]
        print('%-12s' % misc.format_time(time.time() - time_begin), end='')
        for obj, vals in zip(metric_objs, results):
            for val, fmt in zip(vals, obj.get_metric_formatting()):
                print(fmt % val, end='')
        print()
    print()
    
snapshot_pkls = ['model/test_0.pkl','model/test_10.pkl','model/test_90.pkl']
dataset_cfg = {'tfrecord_dir': 'cifar10_wm3000_mullabel', 'max_label_size': 'full', 'verbose': True, 'shuffle_mb': 0}

# metrics=['is']
# evaluate_metrics(snapshot_pkls,dataset_cfg,metrics)    
# metrics=['fid']
# evaluate_metrics(snapshot_pkls,dataset_cfg,metrics) 

Streaming data using dataset.TFRecordDataset...
Dataset shape = [3, 32, 32]
Dynamic range = [0, 255]
Label size    = 10
Initializing metrics.frechet_inception_distance.API...
[3, 32, 32]

Snapshot  Time_eval   FID       
---       ---         ---       
Reals     2m 18s      -0.0000   
90        2m 34s      104.0909  
10        2m 27s      8.1116    
0         2m 35s      8.1207    



## 批量生成图片 

In [24]:
import train
from matplotlib import pyplot as plt
import os
import shutil

training_set = dataset.load_dataset(data_dir=config.data_dir, **dataset_cfg)
print('dataset load finish')

def generate_images(pklpath,training_set):
    tfutil.init_tf(config.tf_config)
    # 载入模型
    with tf.device('/gpu:0'):
    #     if resume_run_id is not None:
            #network_pkl = misc.locate_network_pkl(resume_run_id, resume_snapshot)
        network_pkl = misc.locate_network_pkl(pklpath)
        print('Loading networks from "%s"...' % network_pkl)
        G, D, Gs = misc.load_pkl(network_pkl)
        grid_size, grid_reals, grid_labels, grid_latents = train.setup_snapshot_image_grid(G, training_set, **config.grid)
    # 创建图片文件夹
    path = 'model/quality_assessment/'
    filename = pklpath.split('/')[-1].split('.')[0]
    if os.path.exists(path+filename):
        print('%s文件夹重置'%filename)
        shutil.rmtree(path+filename)
    os.mkdir(path+filename)
    
    # 生成图片
    for i in range(200):
        grid_labels[i] = [1, 0, 0, 0, 1, 1, 0, 0, 0, 1]
        label = 11
        grid_fakes = Gs.run(np.matrix(grid_latents[i]),np.matrix(grid_labels[i]), is_validation=True, randomize_noise=True) 
        misc.save_image_grid(grid_fakes, os.path.join(path+filename+'/fakes%d.png'%(i)), drange=[-1,1], grid_size=[1,1])
    print('%s生成图片成功'%(path+filename))

Streaming data using dataset.TFRecordDataset...
Dataset shape = [3, 32, 32]
Dynamic range = [0, 255]
Label size    = 10
dataset load finish


In [25]:
for pklpath in snapshot_pkls:
    generate_images(pklpath,training_set)

Loading networks from "model/test_0.pkl"...
test_0文件夹重置
model/quality_assessment/test_0生成图片成功
Loading networks from "model/test_10.pkl"...
test_10文件夹重置
model/quality_assessment/test_10生成图片成功
Loading networks from "model/test_90.pkl"...
test_90文件夹重置
model/quality_assessment/test_90生成图片成功
